In [1]:
seed = 161
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json

# Composicion de pipelines
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

# Regresion lineal
from sklearn.linear_model import LinearRegression

# Importar/ Exportar modelos
from joblib import dump, load

# Metricas
from sklearn.metrics import mean_squared_error as mse

# q-q plots
import scipy.stats as stats

import json

In [ ]:
df_conflicto = pd.read_excel("Conflicto Armado.xlsx")
df_conflicto.shape

In [ ]:
df_conflicto.sample(5)

In [ ]:
df_conflicto.dtypes

In [ ]:
df_conflicto.info()
#Dato numerico tiene cerca de 60k nulos
#Dato cualitativo tiene muchos nulos, revisar y si no es rescatable, dropear

In [ ]:
df_conflicto.describe()

In [ ]:
pd.value_counts(df_conflicto['Departamento'])
#Que hago con las de colombia, los dropeo, son pocos

In [ ]:
#Se quita Colombia
def quitarColombia(row):
    if row=="Colombia":
        return None
    return row

df_conflicto['Departamento'] = df_conflicto['Departamento'].apply(quitarColombia)
pd.value_counts(df_conflicto['Departamento'])

In [ ]:
pd.value_counts(df_conflicto['Entidad'])
#nada relevante aqui

In [ ]:
pd.value_counts(df_conflicto['Dimensión'])

In [ ]:
pd.value_counts(df_conflicto['Subcategoría'])
#Los importantes son homicidios por agresor y sexo de la victima
#Presuntos delitos sexuales segun sexo de la victima
#Violencia de pareja
#Violencia de genero

In [ ]:
pd.value_counts(df_conflicto['Dato Numérico'])
#Cada dato numerico debe estar asociado a la unidad de medida asi que no tiene sentido mirar si hay outliers pero se deben transformar en numericos

In [ ]:
#Se arreglan los datos numericos
def arreglarNumerico(row):
    row=str(row).replace('.','')
    row = row.replace(',','.')
    return float(row)

df_conflicto['Dato Numérico'] = df_conflicto['Dato Numérico'].apply(arreglarNumerico)
pd.value_counts(df_conflicto['Dato Numérico'])

In [ ]:
pd.value_counts(df_conflicto['Dato Cualitativo'])
#Que representan los datos cualitativos?
#Se supone que es un intervalo de tiempo. Solo existen 2 intervalos y casi todos los
#demas registros son nulos, la infromacion se considera perdida

In [ ]:
df_conflicto = df_conflicto.drop(['Dato Cualitativo'], axis=1)
#En caso de no poder recuperar los nulos de dato cuantitativo, se pueden dropear

In [ ]:
pd.value_counts(df_conflicto['Año'])
#Los años tienen buen formato

In [ ]:
pd.value_counts(df_conflicto['Mes'])
#Todos los registros son en el ultimo mes, tiene sentido

In [ ]:
pd.value_counts(df_conflicto['Fuente'])

In [ ]:
pd.value_counts(df_conflicto['Unidad de Medida'])

In [ ]:
df_conflicto=df_conflicto.dropna()

In [ ]:
df_depymun = pd.read_csv("Departamentos_y_municipios_de_Colombia.csv")
df_depymun

In [ ]:
pd.value_counts(df_conflicto['Indicador'])

In [ ]:
municipios = {} 

for i, row in df_depymun.iterrows():
    municipio = row['MUNICIPIO'].lower().replace("í","i").replace("á","a").replace("é","e").replace("ó","o").replace("ú","u")
    departamento = row['DEPARTAMENTO'].lower().replace("í","i").replace("á","a").replace("é","e").replace("ó","o").replace("ú","u")
    municipios[municipio] = departamento

In [ ]:
columns=['id','municipio', 'departamento', 'año'] #Este va a tener [id, municipio, departamento, columnasMineria, columnasSalud, columnasConflicto]
columns_pos = {'id':0,'municipio':1, 'departamento':2, 'año':3}

count = 4

columns.append('Violencia hacia la mujer')
columns_pos['Violencia hacia la mujer'] = count
count +=1 

dfinal_mineria = pd.DataFrame(columns=columns)

In [ ]:
jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000
valores_mineria = {}

for i, row in df_conflicto.iterrows():
    municipio = row['Entidad'].split("-")[0].lower().strip()
    departamento = row['Departamento'].lower()
    anio = str(row['Año'])
    llave = municipio + "-" + departamento+"-"+anio
    if llave not in valores_mineria:
      new_row = [llave, municipio, departamento,anio]
      for i in range(0,len(columns)-2):
        new_row.append(0)
      valores_mineria[llave] = new_row
    if row["Indicador"]== "Homicidio de mujeres:  Total" or row["Indicador"]== "Violencia de pareja - Víctima mujer":
        pos = columns_pos['Violencia hacia la mujer']
        valor_viejo = valores_mineria[llave][pos]
        valor_a_sumar=row['Dato Numérico']
        valores_mineria[llave][pos] = valor_viejo + valor_a_sumar
print(columns)
print(valores_mineria)